In [6]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

def Getdata(code_list,page=None):
    # 영화 코드
    # code_list = code_list    
    # 현재 크롤링 중인 영화가 첫 번째 영화인지 여부
    # chk = False
    # 영화의 개수만큼 반복한다.
    df = pd.DataFrame()
    
    for code in code_list:
        # 1단계 : 해당 영화의 평점 페이지 수 계산
        # 접속한다.
        # code= 뒤의 %d를 지우고 원하는 영화코드를 입력해서 웹에서 검색해서 접속한다.
        site1 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=%d&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false' % code
        res1 = requests.get(site1)
        if res1.status_code == requests.codes.ok:
            # 코드를 잘 읽어 왔는지 확인하는 코드
            # html코드를 분석한다.
            bs1 = BeautifulSoup(res1.text, 'html.parser')
            score_total = bs1.find(class_='score_total')
            ems = score_total.find_all('em')
            # print(ems[0].text)
            score_total = int(ems[0].text.replace(',', ''))
            # replace하고 int로 자료형 변경해주어야 함
            print(score_total)
            # 페이지 수를 계산한다.
            pageCnt = score_total // 10
            print(pageCnt)
            # 한 페이지에 10개씩 들어가므로 나머지가 남으면 페이지 수 + 1해준다
            if score_total % 10 > 0:
                pageCnt += 1
            # 현재 페이지 번호(기본 page는 None으로 지정되어 있음)
            if page!=None and pageCnt>=page:
                pageCnt=page
                
            print(pageCnt)
            # 현재 페이지는 첫 번째 페이지로
            now_page = 1

            # 2단계 : 평점 글 정보와 평점을 가져온다.
            while now_page <= pageCnt:
                sleep(0.5) # 잠깐 쉬었다 실행
                
                print('%d / %d' % (now_page, pageCnt))
                # 요청 할 페이지의 주소
                site2 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=%d&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=%d' % (code, now_page)
                res2 = requests.get(site2)
                if res2.status_code == requests.codes.ok:
                    # 코드 잘 읽었는지 확인
                    bs2 = BeautifulSoup(res2.text, 'html.parser')
                    score_result = bs2.find(class_='score_result')
                    lis = score_result.find_all('li')
                    
                    for obj in lis:
                        # 평점
                        star_score = obj.find(class_='star_score')
                        star_em = star_score.find('em')
                        star_score = int(star_em.text)
                        # 평가글
                        score_reple = obj.find(class_='score_reple')
                        reple_p = score_reple.find('span')
                        score_reple = reple_p.text.strip()
                        print(star_score, score_reple)
                        # 데이터를 누적한다
                        if score_reple=='' or score_reple=='관람객':
                            continue
                        else:
                            df = df.append([[score_reple, star_score]], ignore_index=True)
                    now_page += 1
    df.columns = ['text', 'star']
    return df

In [10]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os

def displayWordCloud(code,data = None, backgroundcolor = 'white', width=800, height=600 ):
    # 제외할 단어 지정
    stopwords_kr = ['있었다','것이다'] 
    wordcloud = WordCloud(
                        #font_path = '/Library/Fonts/NanumBarunGothic.ttf', 
                        font_path = '/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/NanumBarunGothic.ttf',
                        stopwords = stopwords_kr,
                        background_color = backgroundcolor, # 기본 인자값으로 white로 받았음. 추후 변경 가능
                        width = width, height = height).generate(data)
    #print(wordcloud.words_)
    # 이미지로 결과 저정
    print(os.getcwd())
    file_dir=os.getcwd()
    file_name="img" + code + ".png"
    wordcloud.to_file(file_dir+"/static/img/"+file_name) 

In [12]:
import os
# vs code와 jupyter상의 디렉토리 경로에 차이가 있음.
print (os.getcwd()) #현재 디렉토리의
print (os.path.realpath(__file__))#파일
print (os.path.dirname(os.path.realpath(__file__)) )#파일이 위치한 디렉토리

# vs code에 해당하는 코드이므로 jupyter에서는 에러 발생(실행 X)

/home/vscode/notebooks/kgitbank-python/03_flask/flask_movie_SQLAlchemy


NameError: name '__file__' is not defined

In [11]:
df1=Getdata([187321],5)
# 첫 번째 인자 리스트로 입력 -> [187321]
# [187321] 영화코드로 영화 관람평을 한 페이지에 5개를 출력해라
df1

4558
455
5
1 / 5
10 관람객
8 충무로: 이거 어케하는거냐?
10 관람객
10 관람객
10 촬영감독의 영혼까지 갈아넣은 마스터피스
10 오스카 작품상 탔어도 할말 없었을것 같다.
10 관람객
10 주인공을 카메라가 계속 따라가는데.. 세트장이 엄청 넓은거에 놀랐습니다. 직접 1차대전에 참전하는 느낌.. 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참.....
10 닥터스트레인지의 판단력이 좋았다.
10 아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음. 감독의 참신성이 빛을 발함. 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이...
2 / 5
10 관람객
10 솔직히 나는 기생충보다 이영화가 더 재미있었다.
10 솔직히 연출력만으로 최고의 영화다. 마치 우리가 투명인간이 되서 그들을 지켜보는 듯했다. 마지막장면은 영화 내내 쉴시간이 없었던것에 대한 피로감을 풀어줘서 기승전결도 완벽.
10 스포일러가 포함된 감상평입니다. 감상평 보기
10 진짜 원테이크 장면 너무 긴거 아님?? 촬영하기 진짜 힘들었을 듯...
10 막판에 횡으로 수많은 병사가 신호와 함께 우르르 튀어나올때 주인공 혼자 종으로 막 달리는 장면 와 진짜ㅋㅋㅋㅋ대박 무조건 아이맥스로 보세요!
10 관람객
10 관람객
10 스포일러가 포함된 감상평입니다. 감상평 보기
9 원테이크 촬영신은 볼때마다 대단하다 후보에 오를만 했음
3 / 5
10 솔직히 샘 멘데스 감독 좀 억울할듯기생충도 재밌게 봤지만, 이거 보고나서 좀 억울했겠다는 생각이 팍듬..
10 관람객
10 일단..극장에서 보는걸  추천드립니다.카메라감독 상 줘야 할판  엄청나게  잘 찍음
10 기생충이 그냥 백인이 나온 헐리우드 영화였다면 절대 1917을 이기지 못했을것. 1917은 오히려 백인의 헐리우드 영화라 차별받음. 진짜 근래 보기 드믄 명작
10 관람객
10 동료를 잃고 목숨을 

,text,star
0,충무로: 이거 어케하는거냐?,8
1,촬영감독의 영혼까지 갈아넣은 마스터피스,10
2,오스카 작품상 탔어도 할말 없었을것 같다.,10
3,주인공을 카메라가 계속 따라가는데.. 세트장이 엄청 넓은거에 놀랐습니다. 직접 1차...,10
4,닥터스트레인지의 판단력이 좋았다.,10
5,아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1...,10
6,솔직히 나는 기생충보다 이영화가 더 재미있었다.,10
7,솔직히 연출력만으로 최고의 영화다. 마치 우리가 투명인간이 되서 그들을 지켜보는 듯...,10
8,스포일러가 포함된 감상평입니다. 감상평 보기,10
9,진짜 원테이크 장면 너무 긴거 아님?? 촬영하기 진짜 힘들었을 듯...,10


In [13]:
displayWordCloud('187321',' '.join(df1['text'])) # 공백을 기준으로 연결하라 (df1의 칼럼인 text를 187321과 연결하라)

/home/vscode/notebooks/python2020/200411_WebCrawling/flask_movie_SQLAlchemy
